In [1]:
import jupyros as jr
import ipywidgets as widgets

from ipywidgets import Button, GridBox, Layout, ButtonStyle, FloatSlider, HBox, VBox

import rospy
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from rt2_assignment1.msg import SetVel
from rt2_assignment1.srv import Command

from IPython.display import display
output = widgets.Output()
vel_msg = SetVel()
lin_init = 0.6
ang_init = 0.6
vel_msg.linear = lin_init
vel_msg.angular = ang_init

# States: 0 (stop), 1 (go_to_point), 2 (directionPad)
from enum import Enum

class Running(Enum):
    STOP = 0
    GTP  = 1
    PAD  = 2
    
running  = Running.STOP
pub_ = None
ui_client = None



In [2]:
class DirectionPad:
    
    def __init__(self, lin_slider, ang_slider):
        self.pub_vel = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        self.twist = Twist()
        self.fwd_m = 0.0
        self.ang_m = 0.0
        self.lin_slider = lin_slider
        self.ang_slider = ang_slider
    
    def forward(self):
        self.fwd_m = 1.0
        self.ang_m = 0.0
    def backward(self):
        self.fwd_m = -1.0
        self.ang_m = 0.0
    def turnCounter(self):
        self.fwd_m = 0.0
        self.ang_m = -1.0
    def turnClock(self):
        self.fwd_m = 0.0
        self.ang_m = 1.0
        
    def pubTwist(self):
        global lin_slider, ang_slider
        self.twist.linear.x = self.fwd_m*lin_slider.value
        self.twist.angular.z = self.ang_m*ang_slider.value
        
        self.pub_vel.publish(self.twist)
        
    def pubStop(self):
        self.twist.linear.x = 0
        self.twist.angular.z = 0
        
        self.pub_vel.publish(self.twist)
    

In [3]:
lin_slider = FloatSlider(description='Linear Velocity', value=lin_init, min=0.1, max=2, step=0.02)
ang_slider = FloatSlider(description='Angular Velocity', value=ang_init, min=0.1, max=2, step=0.02)


v1 = VBox([lin_slider, ang_slider])

In [4]:
dir_pad = DirectionPad(lin_slider, ang_slider)


def lin_vel_change(vel):
    vel_msg.linear = vel["new"]
    pub_.publish(vel_msg)

def ang_vel_change(vel):
    vel_msg.angular = vel["new"]
    pub_.publish(vel_msg)
    
lin_slider.observe(lin_vel_change, names='value')
ang_slider.observe(ang_vel_change, names='value')
    
def start_gotopoint(b):
    global running
    if running != Running.GTP:
        ui_client("start")
        running = Running.GTP
        
def stop_gotopoint(b):
    global running
    if running ==  Running.GTP:
        ui_client("stop")
    elif running == Running.PAD:
        dir_pad.pubStop()
    running =  Running.STOP
        
        
def directionalButton(b):
    global running
    if running == Running.GTP:
        stop_gotopoint(b)
    if b.layout.grid_area == 'up':
        dir_pad.forward()
    elif b.layout.grid_area == 'down':
        dir_pad.backward()
    elif b.layout.grid_area == 'left':
        dir_pad.turnCounter()
    elif b.layout.grid_area == 'right':
        dir_pad.turnClock()
        
    running = Running.PAD
    

def odomCllbck(msg):
    global running
    if running == Running.PAD:
        dir_pad.pubTwist()

In [5]:
    
rospy.init_node("jupyter_interface")


pub_ = rospy.Publisher('/set_vel', SetVel, queue_size=10)
sub = rospy.Subscriber('/odom', Odometry, odomCllbck)

rospy.wait_for_service('/user_interface')
ui_client = rospy.ServiceProxy('/user_interface', Command)

In [6]:


up_b  = Button(description='\u2191', #direction='up',
                 layout=Layout(width='auto', grid_area='up'),
                 style=ButtonStyle(button_color='lightgray'))
down_b    = Button(description='\u2193', #direction='down',
                 layout=Layout(width='auto', grid_area='down'),
                 style=ButtonStyle(button_color='lightgray'))
right_b = Button(description='\u2192', #direction='right',
                 layout=Layout(width='auto', grid_area='right'),
                 style=ButtonStyle(button_color='lightgray'))
left_b  = Button(description='\u2190', #direction='left',
                 layout=Layout(width='auto', grid_area='left'),
                 style=ButtonStyle(button_color='lightgray'))

gb = GridBox(children=[up_b, down_b, right_b, left_b],
        layout=Layout(
            width='100%',
            grid_template_rows='auto auto',
            grid_template_columns='33% 33% 33%',
            grid_template_areas='''
            ". up ."
            "left down right "
            ''')
       )



In [7]:
start_b  = Button(description='START',
                 layout=Layout(width='100%', grid_area='start'),
                 style=ButtonStyle(button_color='lightgray'))
stop_b  = Button(description='STOP',
                 layout=Layout(width='100%', grid_area='stop'),
                 style=ButtonStyle(button_color='lightgray'))

h1 = HBox([start_b, stop_b])

g_all = GridBox(children=[v1, h1, gb],
        layout=Layout(
            width='100%',
            grid_template_rows='auto auto',
            grid_template_columns='33% 40% 27%',
            grid_template_areas='''
            "v1 . gb"
            " . h1 . "
            ''')
       )

In [8]:
start_b.on_click(start_gotopoint)
stop_b.on_click(stop_gotopoint)
up_b.on_click(directionalButton)
down_b.on_click(directionalButton)
left_b.on_click(directionalButton)
right_b.on_click(directionalButton)


In [9]:
display(g_all)
output

GridBox(children=(VBox(children=(FloatSlider(value=0.6, description='Linear Velocity', max=2.0, min=0.1, step=…

Output()

In [ ]:
running